In [1]:
!pip install -q gradio pymupdf sentence-transformers faiss-cpu transformers torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import gradio as gr
import fitz  # PyMuPDF
import pandas as pd
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

In [15]:
# Configuración inicial
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# EMBEDDING_MODEL = "all-MiniLM-L6-v2" # Muy ligero, rápido, baja VRAM, baja precisión. Ideal test rápido
EMBEDDING_MODEL = "all-mpnet-base-v2" # Mejor embeddings, mismo LLM pequeño. Poco más pesado pero mejor calidad embeddings
LLM_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"


In [ ]:
# Carga modelos
modelo_emb = SentenceTransformer(EMBEDDING_MODEL).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
modelo = AutoModelForCausalLM.from_pretrained(LLM_MODEL, torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32).to(DEVICE)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# Variables globales
index = None
df_segmentos = None
document_name = None

In [12]:
# --- Funciones ---

def extraer_segmentos(pdf_bytes):
    """Extrae segmentos de texto relevantes del PDF con mejor limpieza"""
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    segmentos = []

    for pagina in doc:
        texto = pagina.get_text("text")
        # Limpieza más robusta
        for parrafo in texto.split('\n'):
            limpio = ' '.join(parrafo.strip().split())  # Elimina espacios múltiples
            if len(limpio.split()) >= 15:  # Longitud mínima ajustable
                # Dividir párrafos muy largos
                max_words = 150
                words = limpio.split()
                for i in range(0, len(words), max_words):
                    segmento = ' '.join(words[i:i+max_words])
                    segmentos.append(segmento)

    return segmentos

def construir_indice(segmentos):
    """Construye el índice FAISS con normalización de embeddings"""
    embeddings = modelo_emb.encode(segmentos, normalize_embeddings=True)
    index = faiss.IndexFlatIP(embeddings.shape[1])  # Usamos Inner Product para similitud coseno
    index.add(embeddings)
    return index, pd.DataFrame({"texto": segmentos})

def recuperar_contexto(pregunta, k=3, umbral=0.7):
    """Recupera contexto con umbral de similitud"""
    if index is None or df_segmentos is None:
        return None

    emb_pregunta = modelo_emb.encode([pregunta], normalize_embeddings=True)
    distancias, indices = index.search(emb_pregunta, k)

    # Filtra por umbral de similitud
    contextos_relevantes = []
    for i, dist in zip(indices[0], distancias[0]):
        if dist >= umbral:
            contextos_relevantes.append(df_segmentos.iloc[i]["texto"])

    return "\n\n".join(contextos_relevantes) if contextos_relevantes else None

def generar_respuesta(pregunta, contexto):
    """Genera respuesta con prompt mejor estructurado"""
    if not contexto:
        return "No tengo información suficiente en el documento para responder a tu pregunta."

    system_prompt = """Eres un asistente útil que responde preguntas basado únicamente en el contexto proporcionado.
    Contexto:
    {contexto}

    Responde la pregunta de manera precisa y concisa. Si la pregunta no puede ser respondida con el contexto, di exactamente: "No tengo información suficiente para responder a eso".

    Pregunta: {pregunta}
    Respuesta:""".format(contexto=contexto, pregunta=pregunta)

    inputs = tokenizer(system_prompt, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        outputs = modelo.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )

    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extrae solo la parte de la respuesta
    respuesta = respuesta.split("Respuesta:")[-1].strip()
    return respuesta

In [13]:
# --- Interfaz Gradio ---

def cargar_pdf(pdf):
    global index, df_segmentos, document_name
    try:
        with open(pdf.name, "rb") as f:
            contenido = f.read()

        segmentos = extraer_segmentos(contenido)
        if not segmentos:
            return "Error: No se pudo extraer contenido útil del PDF.", ""

        index, df_segmentos = construir_indice(segmentos)
        document_name = pdf.name.split("/")[-1]
        return f" PDF procesado: {document_name} ({len(segmentos)} segmentos)", ""
    except Exception as e:
        return f"Error al procesar PDF: {str(e)}", ""

def consultar(pregunta, k, umbral):
    if not pregunta.strip():
        return "Por favor ingresa una pregunta válida."

    contexto = recuperar_contexto(pregunta, k, umbral/100)  # Convierte de porcentaje a decimal

    # Muestra contexto usado para mayor transparencia
    contexto_info = ""
    if contexto:
        segmentos = contexto.split("\n\n")
        contexto_info = "\n\n Contexto usado:\n" + "\n".join([f"- {s[:100]}..." for s in segmentos[:3]])

    respuesta = generar_respuesta(pregunta, contexto)
    return respuesta + contexto_info

In [14]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""# Chatbot Inteligente para PDFs
    Sube un PDF y haz preguntas sobre su contenido""")

    with gr.Row():
        with gr.Column(scale=1):
            archivo_pdf = gr.File(label="Sube tu PDF", type="filepath")
            boton_cargar = gr.Button("Procesar PDF")
            estado = gr.Textbox(label="Estado", interactive=False)
            gr.Examples(
                examples=["https://arxiv.org/pdf/2307.09288.pdf"],
                inputs=archivo_pdf,
                label="Ejemplos para probar"
            )

        with gr.Column(scale=2):
            entrada = gr.Textbox(label="Tu pregunta", placeholder="Escribe tu pregunta sobre el documento aquí...")
            with gr.Row():
                topk = gr.Slider(1, 5, value=3, step=1, label="N° de fragmentos a considerar")
                umbral = gr.Slider(50, 95, value=70, step=5, label="Umbral de similitud (%)")
            boton_preguntar = gr.Button("Preguntar", variant="primary")
            salida = gr.Textbox(label="Respuesta", interactive=False)

    # Eventos
    boton_cargar.click(cargar_pdf, inputs=archivo_pdf, outputs=[estado, salida])
    boton_preguntar.click(consultar, inputs=[entrada, topk, umbral], outputs=salida)
    entrada.submit(consultar, inputs=[entrada, topk, umbral], outputs=salida)

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ef7f91e3b4338406c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ef7f91e3b4338406c4.gradio.live
